<a href="https://colab.research.google.com/github/nikm7777/D-code/blob/main/Personnel_scheduling_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#function to solve and print tableau with identity readily available
def tableau_solver(A,b,B,C):
  flag=0
  optimal_v=np.dot(C[B],b)  # initial RHS(optimal cost value)
  RC=get_reduced_cost(A,C) # get the initial reduced cost
  while any(x>0 for x in RC):
    ind=np.argmax(RC) #index of maximum reduced cost
    k=np.atleast_2d(A[:,ind]).T #pivot column
    R=b/k
    if any(r>0 for r in R):        #for variables leaving and entering the basis
      pivot=lexicographic_rule(A,k,b,ind) # choosing the pivot row by lexicographic rule for handling degeneracy and prevent cycling
      b[pivot,0]=b[pivot,0]/A[pivot,ind]
      A[pivot,:]=A[pivot,:]/A[pivot,ind]
      i=list(range(0,np.shape(A)[0]))
      i.pop(pivot)
      for index in i:
        multiplier=A[index,ind]
        A[index,:]=A[index,:]-A[pivot,:]*multiplier# making pivot coloumn as identity so it enters the basis
        b[index,0]=b[index,0]-b[pivot,0]*multiplier
      optimal_v=optimal_v-b[pivot,0]*RC[ind]
      RC=RC-A[pivot,:]*RC[ind]  # making max positive reduced cost zero for new basis
      B[pivot]=ind #new basis indices
      print(Optimal_tableau_as_dataframe(A,RC,optimal_v,b,B))

    else:             #if all ratios are negative
      optimal_v='infinite(LP unbounded)' # LP unbounded
      return RC,A,b,B,optimal_v

      flag=1
      break
  if flag==0:
    return RC,A,b,B,optimal_v

In [ ]:
#function to find the basis coloumn indices on the basis of location of identity colomns in A
def find_basis_index(A):
  B=[]
  I=np.identity(np.shape(A)[0])
  for i in range(np.shape(A)[0]):
    for j in range(np.shape(A)[1]):
      if np.array_equal(A[:,j],I[:,i]):
        B.append(j)
        break
  return B

In [ ]:
#Phase 1
#function to introduce Artificial variables(create identity basis) if identity not present
def Auxiliary_var(A):
  I=np.identity(np.shape(A)[0])
  k=[]
  for i in range(np.shape(A)[0]):
    for j in range(np.shape(A)[1]):
      if np.array_equal(A[:,j],I[:,i]):
        k.append(i)
        break
  I=np.delete(I,k,1)
  n=np.shape(I)[1]
  Anew=np.hstack((A,I))
  B=find_basis_index(Anew)
  C=np.concatenate((np.zeros(np.shape(A)[1]),np.ones(np.shape(I)[1])))
  return Anew,B,C,n

In [ ]:
#function to get initial reduced cost
def get_reduced_cost(A,C):
  B=find_basis_index(A)
  N=[]
  for i in range(np.shape(A)[1]):
    if i not in B:
      N.append(i)
  N=np.array(N)
  RC=np.zeros(np.shape(A)[1])
  a=np.dot(C[B],A[:,N])-C[N]
  j=0
  for i in N:
    RC[i]=a[j]
    j+=1
  return RC

In [ ]:
#lexicographic rule to prevent cycling + simple pivoting rule if no degeneracy
def lexicographic_rule(A,k,b,ind):
  r=b/k
  p=np.where(r>=0)[0]
  Atemp=A[p,ind]
  de=np.where(Atemp<0)[0]
  p=np.delete(p,de,axis=0)
  p=np.array(p,dtype=int)
  ri=np.where(r==np.min(r[p]))[0]
  ri=np.array(ri,dtype=int)
  i=0
  while np.shape(ri)[0]>1:
    x=A[ri,i]/A[ri,ind]     # divide by positive y in pivot coloumn
    rtemp=np.where(x==np.min(x))[0]
    rtemp=np.array(rtemp,dtype=int)
    ri=ri[rtemp]   # choose the row with minimum value
    i=i+1
  return ri[0]

In [ ]:
#function to get tableau in the form of dataframe
def Optimal_tableau_as_dataframe(A,RC,optimal_v,b,B):
  RHS=np.vstack((optimal_v,b))
  A2=np.vstack((RC,A))
  Tableau=np.hstack((A2,RHS))
  col_index=[]
  for i in range (np.shape(Tableau)[1]-1):
    col_index.append(f'x{i+1}')
  col_index.append('RHS')
  row_index=['row_0']
  for i in range(len(B)):
    row_index.append(f'x{B[i]+1}')
  Optimal_tableau=pd.DataFrame(data=Tableau,index=row_index,columns=col_index)
  return Optimal_tableau

In [ ]:
# input A,b and C from the LP model(Model 19) in standard form
A=np.zeros((29,42),dtype=float) # array of zeros

#indices where variable coefficient=1
a=[[0,5,9,13],[3,6,10],[1,7,11,14],[4,15,16],[2,8,12,17],[0,1,2],[3,4,19],[5,6,7,8],[9,10,11,12],[13,14,15],[16,17],[0,24],[1,25],[2,26],[3,27],[4,28],[5,29],[6,30],[7,31],[8,32],[9,33],[10,34],[11,35],[12,36],[13,37],[14,38],[15,39],[16,40],[17,41]]
j=0
for i in range(29):
    A[i,a[j]]=1
    j+=1

#for varibale coefficient=-1
j=18

for i in range(5,11):
  A[i,j]=-1         # final A
  j+=1
C=np.array([6,6,6,6.1,6.1,5.9,5.9,5.9,5.9,5.8,5.8,5.8,5.8,6.8,6.8,6.8,7.3,7.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],dtype=float)
b=np.array([14,14,14,14,14,8,8,8,8,7,7,6,6,6,6,6,4,8,4,4,5,5,5,5,3,3,8,6,2],dtype=float)

In [ ]:
# main body of code

b=np.atleast_2d(b).T
B=find_basis_index(A)
print('Simplex iterations:')
temp=0
if len(B)<np.shape(A)[0]: # if identity not present
  print('Phase 1:')       # Phase 1
  Anew,B,Cnew,n=Auxiliary_var(A)  # introduce artificial variables
  RC,A,b,B,optimal_v=tableau_solver(Anew,b,B,Cnew) # modified phase1 problem
  if optimal_v==0:        # if optimal value=0
    print('Phase 2:')     # Phase 2
    n=np.shape(A)[1]-n    # removing artifical variables coloumns
    A=A[:,0:n]
    de=[]
    for i in range(np.shape(A)[0]):
      if all(x==0 for x in A[i,:]) and b[i]==0:  # if artificial variable still in basis i.e in a row and b=0
        print(f'Constraint {i+1} is redundant')   # row is redundant
        de.append(i)
    A=np.delete(A,de,axis=0)         # remove redundant constraint
    b=np.delete(b,de,axis=0)
    for i in de:
        B.pop(i)
    RC,A,b,B,optimal_v=tableau_solver(A,b,B,C)
  else:
    print('LP is infeasible')   #if Phase 1 optimal value!=0, LP is infeasible
    temp=1
else:
  RC,A,b,B,optimal_v=tableau_solver(A,b,B,C)
if temp==0:
  if optimal_v=='infinite(LP unbounded)':     #LP unbounded if solution returned by function is unbounded
    print('LP is unbounded')
  else:
    print('\nOptimal Tableau is:')             # print final optimal tableau
    print(Optimal_tableau_as_dataframe(A,RC,optimal_v,b,B))

Simplex iterations:
Phase 1:
        x1   x2   x3   x4   x5   x6   x7   x8   x9  x10  x11  x12  x13  x14  \
row_0  0.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0   
x43    0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0   
x44    0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
x45    0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
x46    0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x47    0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
x48    0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x49    0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x50    0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
x51    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0   
x52    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
x53    0.0  0.0  0.0  0

<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divid


        x1   x2   x3   x4   x5   x6   x7   x8   x9  x10  x11  x12  x13  x14  \
row_0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  2.0  2.0  2.0  2.0   
x43    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0   
x44    0.0  0.0  0.0  0.0  0.0  0.0  0.0 -1.0 -1.0  0.0  1.0  0.0  0.0  0.0   
x45    0.0  0.0 -1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0   
x46    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x47    0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
x2     0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x5     0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x7     0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
x51    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0   
x52    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
x53    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divid


        x1   x2   x3   x4   x5   x6   x7   x8   x9  x10  x11  x12  x13  x14  \
row_0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x10    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
x11    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
x45    0.0  0.0 -1.0  0.0  0.0  0.0  0.0  0.0 -1.0  0.0  0.0  0.0 -1.0  0.0   
x46    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x47    0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0   
x2     0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x5     0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x7     0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x8     0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
x16    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
x18    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divide
  r=b/k
<ipython-input-12-faa8330cc6ae>:9: RuntimeWarning: divide by zero encountered in true_divide
  R=b/k
<ipython-input-16-e8b39a8ab54e>:3: RuntimeWarning: divide by zero encountered in true_divid

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
Optimal_tableau_as_dataframe(A,RC,optimal_v,b,B) #  print the full optimal tableau as dataframe


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,RHS
row_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,-0.8,-1.3,0.0,0.0,0.0,0.0,-0.1,-0.1,0.0,-0.1,-0.1,-0.2,-0.1,-0.2,-0.2,0.0,0.0,0.0,-8.881784e-16,0.0,429.6
x10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,5.0
x11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,5.0
x21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,11.0
x28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,4.0
x22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.000000e+00,0.0,12.0
x2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,-1.000000e+00,0.0,3.0
x5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,6.0
x7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,7.0
x8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,4.0
